In [50]:
%matplotlib inline
import quantstats as qs
import yfinance as yf
import panel as pn
import pandas as pd
import hvplot.pandas
pn.extension()
import plotly as px
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import HRPOpt
# extend pandas functionality with metrics, etc.
qs.extend_pandas()
window = pn.widgets.IntSlider(name='Window Size', value=6, start=1, end=21)
title = 'Stock'
ticker = pn.widgets.input.TextInput()
# ticker.value = ''
portfolio = None
print(type(portfolio))
init = 0
# input ticker and view stock data

def get_df(ticker, window_size):
    df = yf.Ticker(ticker).history(period="5Y")
    df.reset_index(inplace=True)
    global portfolio
    if ticker is not '' and portfolio is None:
        portfolio = df[['Date', 'Close']]
        portfolio.rename(columns={'Close': ticker}, inplace=True)
    elif ticker is not '':
        portfolio = pd.concat([portfolio, df['Close']], join='inner', axis='columns')
        portfolio.rename(columns={'Close': ticker}, inplace=True)
    
    return df.set_index('Date').rolling(window=window_size).mean().reset_index()
def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return df.hvplot.line('Date', 'Close', grid=True, responsive=True, height=300)
# can include additional plots/visualizations to add to the stock panel by creating additional get_plot functions similiar to the one above. 

pn.Row(
    pn.Column(title, ticker, window),
    pn.bind(get_plot, ticker, window)
)





<class 'NoneType'>
- : No data found for this date range, symbol may be delisted


Row
    [0] Column
        [0] Markdown(str)
        [1] TextInput()
        [2] IntSlider(end=21, name='Window Size', start=1, value=6)
    [1] ParamFunction(function)

In [59]:
# create portfolio analysis
def get_portfolio_opt(portfolio, window_size):
#     portfolio.sort_index(ascending=False, inplace=True)
    try:
        portfolio.set_index(keys='Date', inplace=True)
    except:
        print('index already set')
    print(portfolio)
    returns = portfolio.pct_change().dropna()
    hrp = HRPOpt(returns)
    hrp_weights = hrp.optimize()
    performance = hrp.portfolio_performance(verbose=True)
    return hrp

def get_pie_chart_portfolio():
    hrp = get_portfolio_opt(portfolio, window)
    hrp_weights = hrp.optimize()
    tickers = [i[0] for i in hrp_weights]
    weights = [i[1] for i in hrp_weights]
    df = pd.DataFrame(zip(tickers, weights))
    
    return px.pie(df)
get_portfolio_opt(portfolio, window)
get_pie_chart_portfolio()

index already set
                  MSFT        AAPL         AMD         GME
Date                                                      
2017-02-10   59.505802         NaN   13.580000   20.867582
2017-02-13   60.175243   31.185596   13.490000   20.631744
2017-02-14   60.399742   31.461767   13.260000   20.932638
2017-02-15   60.362335   31.870115   13.300000   20.965170
2017-02-16   60.352974   31.985785   12.970000   20.672409
...                ...         ...         ...         ...
2022-02-03  301.250000  175.616257  120.080002   99.230003
2022-02-04  305.940002  172.679993  123.599998  102.339996
2022-02-07  300.950012  172.389999  123.669998  102.339996
2022-02-08  304.559998  171.660004  128.229996  115.599998
2022-02-09  311.209991  174.830002  132.850006  124.290001

[1259 rows x 4 columns]
Expected annual return: 42.3%
Annual volatility: 19.1%
Sharpe Ratio: 2.11
index already set
                  MSFT        AAPL         AMD         GME
Date                                   

NameError: name 'px' is not defined